In [1]:
from langchain.document_loaders import PyPDFLoader
import time
from pinecone import Pinecone, ServerlessSpec

In [2]:
path = "C:\Workspace\DACNTT\data\TrietHoc\giao_trinh_triet_hoc.pdf"

# Đọc PDF từng trang
loader = PyPDFLoader(path)
documents = loader.load()

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\dhdnc\AppData\Local\Temp\ipykernel_16436\3835062692.py:1: SyntaxWarning: invalid escape sequence '\W'
  path = "C:\Workspace\DACNTT\data\TrietHoc\giao_trinh_triet_hoc.pdf"


In [3]:
documents

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2021-06-09T11:20:31+07:00', 'author': 'HP', 'moddate': '2021-06-09T11:20:31+07:00', 'source': 'C:\\Workspace\\DACNTT\\data\\TrietHoc\\giao_trinh_triet_hoc.pdf', 'total_pages': 258, 'page': 0, 'page_label': '1'}, page_content='HỘI ĐỒNG BIÊN SOẠN GIÁO TRÌNH  \nMÔN TRIẾT HỌC MÁC - LÊNIN \n \n \n \n \n \n \n \n \n \n \nGIÁO TRÌNH \n \nTRIẾT HỌC MÁC - LÊNIN \n \n \nTrình độ: Đại học  \nĐối tượng: Khối các ngành ngoài lý luận chính trị \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nHÀ NỘI - 2019'),
 Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2021-06-09T11:20:31+07:00', 'author': 'HP', 'moddate': '2021-06-09T11:20:31+07:00', 'source': 'C:\\Workspace\\DACNTT\\data\\TrietHoc\\giao_trinh_triet_hoc.pdf', 'total_pages': 258, 'page': 1, 'page_label': '2'}, page_

# Gọi mô hình Embed

In [4]:
from sentence_transformers import SentenceTransformer
# from sentence_transformers.util import cos_sim

sentences = ['Xin chào các bạn']

model = SentenceTransformer('Alibaba-NLP/gte-multilingual-base', trust_remote_code=True)
embeddings = model.encode(sentences)
# print(cos_sim(embeddings[0], embeddings[1]))

c:\Workspace\DACNTT\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
print(embeddings)

[[-5.64131662e-02  6.88597932e-02 -8.35299715e-02  3.58179361e-02
  -1.88229792e-02 -2.84741279e-02 -6.01125807e-02  9.30784047e-02
   6.85936883e-02 -5.31905666e-02  1.05727306e-02  3.50336619e-02
  -9.14612785e-02  5.76380119e-02 -6.63745180e-02  5.02175801e-02
   1.47357687e-01  5.29701635e-02 -1.31218163e-02 -2.25168727e-02
  -7.61575159e-03 -2.15137582e-02 -4.41560932e-02  4.78426889e-02
  -3.35712954e-02  4.53836285e-02  8.59050080e-02 -1.95835307e-02
  -6.48251399e-02 -2.67114677e-02 -9.26883221e-02 -1.24890264e-02
   4.46201861e-03  4.09556516e-02  4.61970940e-02 -8.20328519e-02
  -3.85879613e-02  2.26650164e-02 -9.29855835e-03 -2.18419041e-02
  -6.54402971e-02 -4.71034609e-02 -1.47112384e-02  2.50825775e-03
   2.37255841e-02  1.03411963e-02  7.29774013e-02  3.94185670e-02
   1.54358055e-02  2.27280576e-02  1.93398297e-02 -5.43419737e-03
   2.83450801e-02  4.47928794e-02  5.21918349e-02  3.02956477e-02
  -7.22458214e-02  4.35438119e-02  6.77041560e-02  1.08460318e-02
   2.25924

# Embed biên soạn Triết


In [6]:
triet_hoc_records = []

content = "HỘI ĐỒNG BIÊN SOẠN\nGS. TS. Phạm Văn Đức (chủ biên)  \nGS. TS. Trần Văn Phòng  \nPGS. TS. Nguyễn Tài Đông  \nThiếu tướng GS. TS. Nguyễn Văn Tài  \nGS. TS. Nguyễn Trọng Chuẩn  \nGS. TS. Hồ Sĩ Quý  \nPGS. TSKH. Lương Đình Hải  \nPGS. TS. Nguyễn Anh Tuấn  \nPGS. TS. Trần Đăng Sinh \nCỘNG TÁC BIÊN SOẠN \nThiếu tướng GS. TS.Trương GiangLong  \nGS. TS. Trần Phúc Thăng  \nGS. TS. Nguyễn Hùng Hậu "
embedding = model.encode(sentences)

first_chunk = {
    "id": "0",
    "values": embedding[0].tolist(),
    "metadata": {
        "subject": "TRIẾT HỌC MÁC - LÊNIN ",
        "chapter": "None",
        "chap_context": "None",
        "header": "None",
        "header_context": "HỘI ĐỒNG BIÊN SOẠN GIÁO TRÌNH ",
        "chunk_text": "HỘI ĐỒNG BIÊN SOẠN\nGS. TS. Phạm Văn Đức (chủ biên)  \nGS. TS. Trần Văn Phòng  \nPGS. TS. Nguyễn Tài Đông  \nThiếu tướng GS. TS. Nguyễn Văn Tài  \nGS. TS. Nguyễn Trọng Chuẩn  \nGS. TS. Hồ Sĩ Quý  \nPGS. TSKH. Lương Đình Hải  \nPGS. TS. Nguyễn Anh Tuấn  \nPGS. TS. Trần Đăng Sinh \nCỘNG TÁC BIÊN SOẠN \nThiếu tướng GS. TS.Trương GiangLong  \nGS. TS. Trần Phúc Thăng  \nGS. TS. Nguyễn Hùng Hậu "
    },
}

triet_hoc_records.append(first_chunk)

In [7]:
len(embedding[0])

768

In [8]:
triet_hoc_records[0]

{'id': '0',
 'values': [-0.05641316622495651,
  0.06885979324579239,
  -0.08352997153997421,
  0.035817936062812805,
  -0.01882297918200493,
  -0.028474127873778343,
  -0.06011258065700531,
  0.09307840466499329,
  0.06859368830919266,
  -0.05319056659936905,
  0.01057273056358099,
  0.035033661872148514,
  -0.09146127849817276,
  0.0576380118727684,
  -0.0663745179772377,
  0.050217580050230026,
  0.14735768735408783,
  0.05297016352415085,
  -0.013121816329658031,
  -0.022516872733831406,
  -0.007615751586854458,
  -0.02151375822722912,
  -0.044156093150377274,
  0.047842688858509064,
  -0.03357129544019699,
  0.04538362845778465,
  0.08590500801801682,
  -0.019583530724048615,
  -0.064825139939785,
  -0.026711467653512955,
  -0.09268832206726074,
  -0.01248902641236782,
  0.004462018609046936,
  0.04095565155148506,
  0.04619709402322769,
  -0.08203285187482834,
  -0.038587961345911026,
  0.02266501635313034,
  -0.009298558346927166,
  -0.021841904148459435,
  -0.06544029712677002,


# Upsert phần biên soạn vào Pinecone

In [ ]:
pc = Pinecone(api_key="*")

In [10]:
pc.list_indexes()
#     rag-general-subject

[
    {
        "name": "movies-walkthrough",
        "metric": "cosine",
        "host": "movies-walkthrough-y0c06t1.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1024,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "test-index",
        "metric": "cosine",
        "host": "test-index-y0c06t1.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1024,
        "deletion_protection": "disabled",
        "tags": {
          

In [11]:
index = pc.Index("rag-general-subject")
index.upsert(namespace='triet-hoc', vectors=triet_hoc_records)

{'upserted_count': 1}

# ***Chunking các nội dung chính***


In [12]:
documents[4::]

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2021-06-09T11:20:31+07:00', 'author': 'HP', 'moddate': '2021-06-09T11:20:31+07:00', 'source': 'C:\\Workspace\\DACNTT\\data\\TrietHoc\\giao_trinh_triet_hoc.pdf', 'total_pages': 258, 'page': 4, 'page_label': '5'}, page_content='CHƯƠNG I \n \nTRIẾT HỌC VÀ VAI TRÒ CỦA TRIẾT HỌC  \nTRONG ĐỜI SỐNG XÃ HỘI \n \nI. TRIẾT HỌC VÀ VẤN ĐỀ CƠ BẢN CỦA TRIẾT HỌC \n \n1. Khái lược về triết học \n \n1.1. Nguồn gốc của triết học \n \nLà một loại hình nhận thức đặc thù của con người, triết học ra đời ở cả \nPhương Đông và Phương Tây gần như cùng m ột thời gian (khoảng từ thế kỷ \nVIII đến thế kỷ VI tr.CN) tại các trung tâm văn minh lớn của nhân loại thời Cổ \nđại. Ý thức triết học xuất hiện không ngẫu nhiên, mà có nguồn gốc thực tế từ \ntồn tại xã hội với một trình độ nhất định của sự phát triển văn minh, vă n hóa và \nkhoa học. Con người, với kỳ vọng được đáp ứng nhu cầu về nhận thức và hoạt \nđộng

In [13]:
import re
from langchain.schema import Document

def split_by_heading(documents, heading_regex=r"^(PHẦN|CHƯƠNG|I\.|1\.|1\.1)", min_length=100):
    chunks = []
    for doc in documents:
        content = doc.page_content
        lines = content.split("\n")

        current_chunk = ""
        current_header = "No header"
        for line in lines:
            if re.match(heading_regex, line.strip(), flags=re.IGNORECASE):
                # Lưu chunk cũ nếu đủ dài
                if len(current_chunk.strip()) > min_length:
                    chunks.append(Document(
                        page_content=current_chunk.strip(),
                        metadata={"header": current_header, "source": doc.metadata.get("source"), "page": doc.metadata.get("page")}
                    ))
                # Bắt đầu chunk mới
                current_header = line.strip()
                current_chunk = ""
            current_chunk += line + "\n"

        # Lưu đoạn cuối
        if len(current_chunk.strip()) > min_length:
            chunks.append(Document(
                page_content=current_chunk.strip(),
                metadata={"header": current_header, "source": doc.metadata.get("source"), "page": doc.metadata.get("page")}
            ))
    return chunks


In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["CHƯƠNG 1", "CHƯƠNG 2", "CHƯƠNG 3"]
)

chunks = text_splitter.split_text(documents)

TypeError: expected string or bytes-like object, got 'list'